In [ ]:
import findspark
findspark.init()
import pyspark
import os

sc = pyspark.SparkContext()
spark = pyspark.SQLContext(sc)

working_dir = os.getcwd()

In [1]:
# Load parquet file into dataframe
df = spark.read.parquet("file://" + working_dir + "/testDataset/part-00000-711fabb0-5efc-4d83-afad-0e03a3156794.snappy.parquet")

In [8]:
# Count lines
df.count()

600000

In [59]:
# Show first 5 lines
df.show(5)

+--------------------+--------------------+-------------------+---------+--------+---------+
|            deviceId|             eventId|          eventTime|      lat|     lon|eventType|
+--------------------+--------------------+-------------------+---------+--------+---------+
|8f88ac99-8aa5-88a...|000c5bd2-d2d5-4cf...|2017-05-24 12:34:40|47.409271|8.547062| location|
|8f88ac99-8aa5-88a...|001b3982-ec22-4bb...|2017-05-24 16:21:41|47.417977|8.554384| location|
|8f88ac99-8aa5-88a...|00287463-b763-40b...|2017-05-24 16:02:50|47.416406| 8.55298| location|
|8f88ac99-8aa5-88a...|002946bb-19b8-40a...|2017-05-24 15:03:39|47.417743|8.554184| location|
|8f88ac99-8aa5-88a...|002947fd-bb93-434...|2017-05-24 12:56:02|47.406545|8.547231| location|
+--------------------+--------------------+-------------------+---------+--------+---------+
only showing top 5 rows



In [16]:
# Remove rows with any null or NaN value
df.dropna

<bound method DataFrameNaFunctions.drop of <pyspark.sql.dataframe.DataFrameNaFunctions object at 0x10c8e7f98>>

In [17]:
# Create Temp View from the dataset
df.createOrReplaceTempView("dataset")

In [64]:
# Create a view with only the values we need for processing
spark.sql("SELECT deviceId, eventTime, lat, lon, eventType FROM dataset WHERE (eventType='trip-start' OR eventType='trip-end') \
AND (lat != 0.0 and lon != 0.0) ORDER BY eventTime").createOrReplaceTempView("cleands")


In [113]:
# Show an example of data for a certain device
singleDevice = spark.sql("SELECT * FROM cleands WHERE deviceId='D08699AE-BDAC-4AB4-2F15-177C74993133'")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100, False)

+------------------------------------+-------------------+---------+----------+----------+
|deviceId                            |eventTime          |lat      |lon       |eventType |
+------------------------------------+-------------------+---------+----------+----------+
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-25 23:46:05|32.988837|-97.263735|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-25 23:46:09|32.988837|-97.263735|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 00:35:30|32.98925 |-97.263821|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 00:35:39|32.989253|-97.263822|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 02:08:40|32.841525|-97.068658|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 02:08:46|32.841525|-97.068658|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 13:57:30|32.926142|-97.087224|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 19:17:41|32.98928 |-97.264482|trip-start|

In [114]:
# Shift values of eventType to add previous and next event
singleDevice = spark.sql("SELECT *, LAG(eventType, 1) OVER (ORDER BY eventTime) AS previousEvent, \
LEAD(eventType, 1) OVER (ORDER BY eventTime) AS nextEvent FROM singledevice")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100)

+--------------------+-------------------+---------+----------+----------+-------------+----------+
|            deviceId|          eventTime|      lat|       lon| eventType|previousEvent| nextEvent|
+--------------------+-------------------+---------+----------+----------+-------------+----------+
|D08699AE-BDAC-4AB...|2017-05-25 23:46:05|32.988837|-97.263735|  trip-end|         null|  trip-end|
|D08699AE-BDAC-4AB...|2017-05-25 23:46:09|32.988837|-97.263735|  trip-end|     trip-end|  trip-end|
|D08699AE-BDAC-4AB...|2017-05-26 00:35:30| 32.98925|-97.263821|  trip-end|     trip-end|trip-start|
|D08699AE-BDAC-4AB...|2017-05-26 00:35:39|32.989253|-97.263822|trip-start|     trip-end|  trip-end|
|D08699AE-BDAC-4AB...|2017-05-26 02:08:40|32.841525|-97.068658|  trip-end|   trip-start|  trip-end|
|D08699AE-BDAC-4AB...|2017-05-26 02:08:46|32.841525|-97.068658|  trip-end|     trip-end|trip-start|
|D08699AE-BDAC-4AB...|2017-05-26 13:57:30|32.926142|-97.087224|trip-start|     trip-end|trip-start|


In [116]:
singleDevice = spark.sql("SELECT deviceId, eventTime, lat, lon, eventType FROM singledevice \
WHERE NOT ((eventType='trip-end' AND previousEvent='trip-end') \
OR (eventType='trip-start' AND nextEvent='trip-start'))")
singleDevice.createOrReplaceTempView("singledevice")
singleDevice.show(100, False)


+------------------------------------+-------------------+---------+----------+----------+
|deviceId                            |eventTime          |lat      |lon       |eventType |
+------------------------------------+-------------------+---------+----------+----------+
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 00:35:39|32.989253|-97.263822|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 02:08:40|32.841525|-97.068658|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 19:17:41|32.98928 |-97.264482|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-05-26 19:19:30|32.988822|-97.263723|trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-06-01 00:11:22|32.989214|-97.26381 |trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-06-01 00:38:47|32.841445|-97.06854 |trip-end  |
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-06-01 13:44:54|32.789698|-97.133493|trip-start|
|D08699AE-BDAC-4AB4-2F15-177C74993133|2017-06-02 01:17:43|32.84152 |-97.068662|trip-end  |